# Define Area of Interest on Google Earth Engine

Earth Engine Explorer (EE Explorer) is a lightweight geospatial image data viewer with access to a large set of global and regional datasets available in the Earth Engine Data Catalog (https://developers.google.com/earth-engine/datasets/). It allows for quick viewing of data with the ability to zoom and pan anywhere on Earth, adjust visualization settings, and layer data to inspect change over time.

* In this notebook, users select a shape from a layer saved in the Earth Engine Data Catalog that defines model extent. This will be the region that is used to clip all raster files and ultimately dictate the region that is run in Biome-BGC Muso.

<div class="alert alert-block alert-danger">
<b> Project Name Setting: </b> Please set the Project Name and Cell Resolution for your project </div>

In [15]:
# CHANGE PROJECT NAME HERE
PROJECT_NAME = "Spout_Run"

# CURRENT DIRECTORY HERE
CURRENT_DIRECTORY = "/scratch/tmb4ee/MODEL_62"

In [16]:
import os, shutil
os.chdir(CURRENT_DIRECTORY)

In [17]:
# Define folder paths as global variables
PROJECT_DIR = os.path.join(CURRENT_DIRECTORY, PROJECT_NAME)
RAWGIS_DIR = os.path.join(PROJECT_DIR, "gis_data")
RAWOBS_DIR = os.path.join(PROJECT_DIR, "obs")
RAWSOIL_DIR = os.path.join(RAWGIS_DIR, "soil")
MODEL_DIR = os.path.join(PROJECT_DIR, 'model')
DEF_DIR = os.path.join(MODEL_DIR, 'defs')
INI_DIR = os.path.join(MODEL_DIR, 'ini_files')
EPC_DIR = os.path.join(MODEL_DIR, 'epc_files')
OUTPUT_DIR = os.path.join(MODEL_DIR, 'output')
CO2_DIR = os.path.join(MODEL_DIR, 'co2')
NDEP_DIR = os.path.join(MODEL_DIR, 'ndep')
ENDPOINT_DIR = os.path.join(MODEL_DIR, 'endpoint_files')
SPINUP_DIR = os.path.join(MODEL_DIR, 'spinup')
NORMAL_DIR = os.path.join(MODEL_DIR, 'normal')
MODEL_RAST_DIR = os.path.join(MODEL_DIR, 'raster_inputs')
IMAGE = os.path.join(PROJECT_DIR, 'image_map')

## Install geemap package

In [18]:
# Installs geemap package
import sys, site

try:
    import geemap
except ImportError:
    # If error with this cell, try commenting out next line and un-commenting two lines below.
    !pip install -U geemap
    # !{sys.executable} -m pip install geemap==0.9.1
    sys.path.insert(0, site.getusersitepackages())

# Authenticates and initializes Earth Engine
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

## Create an interactive map

geemap is a Python package that allows users to interactively interact with datasets stored in the Google Earth Engine data repository. Below, we create an empty map using geemap and select an area of interest by selecting a data layer and feature. Several easy options are given below in layer_dict.

In [19]:
Map = geemap.Map(center=[40,-100], zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Define AOI Using Polygon from Available Library

<div class="alert alert-block alert-danger">
<b> Manual Input: </b> Please specify the layer in which you want to use a boundary from.</div>

In [20]:
BOUNDARY_TYPE = 'huc12'

In [21]:
layer_dict = {
    'States': ["TIGER/2018/States", "NAME"],
    'Counties': ["TIGER/2018/Counties", "NAME", "STATEFP"],
    'huc2': ["USGS/WBD/2017/HUC02", "huc2"],
    'huc4': ["USGS/WBD/2017/HUC04", "huc4"],
    'huc6': ["USGS/WBD/2017/HUC06", "huc6"],
    'huc8': ["USGS/WBD/2017/HUC08", "huc8"],
    'huc10': ["USGS/WBD/2017/HUC10", "huc10"],
    'huc12': ["USGS/WBD/2017/HUC12", "huc12"]
}
boundary_layer = ee.FeatureCollection(layer_dict[BOUNDARY_TYPE][0])
Map.addLayer(boundary_layer, {}, BOUNDARY_TYPE)

<div class="alert alert-block alert-danger">
<b> Manual Input: </b> Please specify boundary feature name and state code if applicatble.</div>

In [22]:
FEATURE = '020700070105'

# Specify STATE_FP Code if wanting AOI as a county!
STATE_FP = '51'

In [23]:
if BOUNDARY_TYPE != "Counties":
    boundary_layer = ee.FeatureCollection(layer_dict[BOUNDARY_TYPE][0]).filter(ee.Filter.eq(layer_dict[BOUNDARY_TYPE][1], FEATURE))
elif BOUNDARY_TYPE == "Counties":
    boundary_layer = ee.FeatureCollection(layer_dict[BOUNDARY_TYPE][0]).filter(ee.Filter.eq(layer_dict[BOUNDARY_TYPE][1], FEATURE)).filter(ee.Filter.eq(layer_dict[BOUNDARY_TYPE][2], STATE_FP))
    
Map.addLayer(boundary_layer, {}, BOUNDARY_TYPE)

In [24]:
# Export area of interest to shapefile
geemap.ee_to_shp(boundary_layer, os.path.join(RAWGIS_DIR, 'AOI.shp'))

Generating URL ...
Please wait ...
Data downloaded to /scratch/tmb4ee/MODEL_62/Spout_Run/gis_data/AOI.shp
